In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Reference To Refer To

This is the link I am using as a guide

https://github.com/swar/nba_api/blob/master/docs/examples/PlayByPlay.ipynb

General workflow so far seems:

1) Get all the games that have been played
2) Scrape their id
3) Do something

Once the data is scraped one thing to keep an eye for is what things stand for:

EventMsgType Key

FIELD_GOAL_MADE = 1
FIELD_GOAL_MISSED = 2
FREE_THROWfree_throw_attempt = 3
REBOUND = 4
TURNOVER = 5
FOUL = 6
VIOLATION = 7
SUBSTITUTION = 8
TIMEOUT = 9
JUMP_BALL = 10
EJECTION = 11
PERIOD_BEGIN = 12
PERIOD_END = 13

Subs are recorded as SUB:Last Name1 FOR Last Name2 and full under EventMsgType 8
The event goes in the description for whichever team underwent the event

Fouls are recorded as Fouler Last Name1 P.FOUL (P1.T1) (Foulee Last Name) where P1 says one personal foul so far. The event goes in the description for whichever team commited the foul


# Explore Data Set Composition For One Game

Procedure To Pull Data

In [2]:
# Get the data for the team in question
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

# Select the dictionary for the Pacers, which contains their team ID
pacers = [team for team in nba_teams if team['abbreviation'] == 'IND'][0]
pacers_id = pacers['id']
print(f'pacers_id: {pacers_id}')

pacers_id: 1610612754


In [3]:
# Now I can use the team ID to pick a game's data
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=pacers_id,
                                                season_nullable=Season.default,
                                                season_type_nullable=SeasonType.regular)

games_dict = gamefinder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']
# Games will have every game the Pacers played this season

# For example sake get the most recent game
game = games[0]
game_id = game['GAME_ID']
print(f'game_id: {game_id}')

game_id: 0022300019


In [4]:
# Now I can query the play by play data for that game
from nba_api.stats.endpoints import playbyplay
df = playbyplay.PlayByPlay(game_id).get_data_frames()[0]
df.head()

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

Explore How the Data is Recorded

In [ ]:
# Now find how substitutions are recorded
substitutions = df[df['EVENTMSGTYPE'] == 8] 
substitutions.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
36,0022300019,54,8,0,1,7:20 PM,7:45,None,None,SUB: Smith FOR Turner,None,None
44,0022300019,69,8,0,1,7:21 PM,7:05,None,None,SUB: Hield FOR Haliburton,None,None
45,0022300019,70,8,0,1,7:21 PM,7:05,None,None,SUB: McConnell FOR Mathurin,None,None
46,0022300019,71,8,0,1,7:21 PM,7:05,None,None,SUB: Nesmith FOR Brown,None,None
60,0022300019,95,8,0,1,7:24 PM,5:42,None,None,SUB: Haliburton FOR Toppin,None,None


In [ ]:
# Now see how fouls are recorded
fouls = df[df['EVENTMSGTYPE'] == 6]
fouls.head() 

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
7,0022300019,12,6,3,1,7:12 PM,11:16,Embiid L.B.FOUL (P1.T1) (S.Wall),None,None,None,None
30,0022300019,45,6,1,1,7:18 PM,8:19,None,None,Turner P.FOUL (P1.T1) (S.Wall),None,None
34,0022300019,51,6,2,1,7:19 PM,7:45,None,None,Turner S.FOUL (P2.T2) (J.Goldenberg),None,None
41,0022300019,65,6,2,1,7:21 PM,7:05,Harris S.FOUL (P1.T2) (J.Goldenberg),None,None,None,None
49,0022300019,79,6,1,1,7:22 PM,6:30,None,None,Smith P.FOUL (P1.T3) (J.Goldenberg),None,None


In [ ]:
# Now I want to see the order of events after fouls

# Filter the DataFrame for fouls
foul_events = df[df['EVENTMSGTYPE'] == 6]

# Get the indices of the foul events
foul_indices = foul_events.index

# Get the indices of the foul event and the next 5 events after each foul
post_foul_indices = [[index + i for i in range(6) if index + i in df.index] for index in foul_indices]

# Flatten the list of lists of indices
post_foul_indices = [index for sublist in post_foul_indices for index in sublist]

# Use iloc to get the rows corresponding to the post-foul events, and print the result
df.iloc[post_foul_indices].head(10)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
7,0022300019,12,6,3,1,7:12 PM,11:16,Embiid L.B.FOUL (P1.T1) (S.Wall),None,None,None,None
8,0022300019,14,2,79,1,7:12 PM,11:02,Maxey BLOCK (1 BLK),None,MISS Brown 11' Pullup Jump Shot,None,None
9,0022300019,16,4,0,1,7:12 PM,10:59,Harris REBOUND (Off:0 Def:1),None,None,None,None
10,0022300019,17,1,41,1,7:12 PM,10:55,Harris 1' Running Layup (2 PTS),None,None,3 - 2,-1
11,0022300019,18,1,101,1,7:13 PM,10:44,None,None,Toppin 7' Driving Floating Jump Shot (2 PTS) (...,5 - 2,-3
12,0022300019,20,2,80,1,7:13 PM,10:31,MISS Embiid 29' 3PT Step Back Jump Shot,None,None,None,None
30,0022300019,45,6,1,1,7:18 PM,8:19,None,None,Turner P.FOUL (P1.T1) (S.Wall),None,None
31,0022300019,47,1,108,1,7:19 PM,8:09,Harris 1' Cutting Dunk Shot (8 PTS) (Embiid 1 ...,None,None,14 - 8,-6
32,0022300019,49,2,80,1,7:19 PM,7:53,None,None,MISS Turner 27' 3PT Step Back Jump Shot,None,None
33,0022300019,50,4,0,1,7:19 PM,7:51,Maxey REBOUND (Off:0 Def:1),None,None,None,None


Establish a procedure on how the data will be collected

In [ ]:
# It appears that the best way to see what happens after a foul is to look at what happens in the same 
# PERIOD and PCTIMESTRING. This is because the EVENTNUM is not always sequential.

# Create a DataFrame of what happens at the time of each foul, so when a foul occurs, 
# see what else happens at the same PERIOD and PCTIMESTRING and store it in a DataFrame
# Extract rows where EVENTMSGTYPE is 6 (foul events)
foul_times = df[df['EVENTMSGTYPE'] == 6][['PERIOD', 'PCTIMESTRING']]

# Merge the original DataFrame with the foul_times DataFrame on 'PERIOD' and 'PCTIMESTRING'
foul_plays = pd.merge(df, foul_times, on=['PERIOD', 'PCTIMESTRING'], how='inner')

# Display foul_plays DataFrame
foul_plays.head(15)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0022300019,10,2,47,1,7:12 PM,11:16,MISS Embiid 6' Turnaround Jump Shot,None,None,None,None
1,0022300019,11,4,0,1,7:12 PM,11:16,None,None,Pacers Rebound,None,None
2,0022300019,12,6,3,1,7:12 PM,11:16,Embiid L.B.FOUL (P1.T1) (S.Wall),None,None,None,None
3,0022300019,45,6,1,1,7:18 PM,8:19,None,None,Turner P.FOUL (P1.T1) (S.Wall),None,None
4,0022300019,51,6,2,1,7:19 PM,7:45,None,None,Turner S.FOUL (P2.T2) (J.Goldenberg),None,None
5,0022300019,53,3,11,1,7:20 PM,7:45,Embiid Free Throw 1 of 2 (1 PTS),None,None,14 - 9,-5
6,0022300019,54,8,0,1,7:20 PM,7:45,None,None,SUB: Smith FOR Turner,None,None
7,0022300019,56,3,12,1,7:20 PM,7:45,Embiid Free Throw 2 of 2 (2 PTS),None,None,14 - 10,-4
8,0022300019,63,1,98,1,7:21 PM,7:05,None,None,Smith 4' Cutting Layup Shot (2 PTS) (Haliburto...,18 - 13,-5
9,0022300019,65,6,2,1,7:21 PM,7:05,Harris S.FOUL (P1.T2) (J.Goldenberg),None,None,None,None


The Points of Analysis Are:
1) What does a team do after each foul?
2) What’s the probability a team pulls their player after a foul?
3) What variables go into this decision?

I Need To Revise How the Data Is Stored To Suit the Context of My Problem

Feature Engineering Around the Metrics Related to Fouls

In [ ]:
# Create a column called fouler which contains the player who committed the foul
# Fill that by seeing the text that comes before the word '<char>.FOUL' in the 'HOMEDESCRIPTION' 
# and 'VISITORDESCRIPTION' columns
foul_plays['fouler'] = np.where(foul_plays['HOMEDESCRIPTION'].str.contains('.FOUL'), 
                                foul_plays['HOMEDESCRIPTION'].str.split(' ').str[0], 
                                np.where(foul_plays['VISITORDESCRIPTION'].str.contains('.FOUL'), 
                                         foul_plays['VISITORDESCRIPTION'].str.split(' ').str[0], 
                                         np.nan))

foul_plays.head(15)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,fouler
0,0022300019,10,2,47,1,7:12 PM,11:16,MISS Embiid 6' Turnaround Jump Shot,None,None,None,None,NaN
1,0022300019,11,4,0,1,7:12 PM,11:16,None,None,Pacers Rebound,None,None,NaN
2,0022300019,12,6,3,1,7:12 PM,11:16,Embiid L.B.FOUL (P1.T1) (S.Wall),None,None,None,None,Embiid
3,0022300019,45,6,1,1,7:18 PM,8:19,None,None,Turner P.FOUL (P1.T1) (S.Wall),None,None,Turner
4,0022300019,51,6,2,1,7:19 PM,7:45,None,None,Turner S.FOUL (P2.T2) (J.Goldenberg),None,None,Turner
5,0022300019,53,3,11,1,7:20 PM,7:45,Embiid Free Throw 1 of 2 (1 PTS),None,None,14 - 9,-5,NaN
6,0022300019,54,8,0,1,7:20 PM,7:45,None,None,SUB: Smith FOR Turner,None,None,NaN
7,0022300019,56,3,12,1,7:20 PM,7:45,Embiid Free Throw 2 of 2 (2 PTS),None,None,14 - 10,-4,NaN
8,0022300019,63,1,98,1,7:21 PM,7:05,None,None,Smith 4' Cutting Layup Shot (2 PTS) (Haliburto...,18 - 13,-5,NaN
9,0022300019,65,6,2,1,7:21 PM,7:05,Harris S.FOUL (P1.T2) (J.Goldenberg),None,None,None,None,Harris


In [ ]:
# Create a foul_counts DataFrame which contains the number of fouls each player committed
# Example: Turner S.FOUL (P2.T2) (J.Goldenberg) here Turner committed a foul
# he has 2 fouls (because P2) and the team has 2 fouls (because T2)
foul_plays['foul_counts'] = foul_plays[['HOMEDESCRIPTION', 'VISITORDESCRIPTION']].apply(
    lambda row: row.str.extract(r'\(P(\d+)\.').astype(float).sum(), axis=1
)

foul_plays.head(15)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,fouler,foul_counts
0,0022300019,10,2,47,1,7:12 PM,11:16,MISS Embiid 6' Turnaround Jump Shot,None,None,None,None,NaN,0.0
1,0022300019,11,4,0,1,7:12 PM,11:16,None,None,Pacers Rebound,None,None,NaN,0.0
2,0022300019,12,6,3,1,7:12 PM,11:16,Embiid L.B.FOUL (P1.T1) (S.Wall),None,None,None,None,Embiid,1.0
3,0022300019,45,6,1,1,7:18 PM,8:19,None,None,Turner P.FOUL (P1.T1) (S.Wall),None,None,Turner,1.0
4,0022300019,51,6,2,1,7:19 PM,7:45,None,None,Turner S.FOUL (P2.T2) (J.Goldenberg),None,None,Turner,2.0
5,0022300019,53,3,11,1,7:20 PM,7:45,Embiid Free Throw 1 of 2 (1 PTS),None,None,14 - 9,-5,NaN,0.0
6,0022300019,54,8,0,1,7:20 PM,7:45,None,None,SUB: Smith FOR Turner,None,None,NaN,0.0
7,0022300019,56,3,12,1,7:20 PM,7:45,Embiid Free Throw 2 of 2 (2 PTS),None,None,14 - 10,-4,NaN,0.0
8,0022300019,63,1,98,1,7:21 PM,7:05,None,None,Smith 4' Cutting Layup Shot (2 PTS) (Haliburto...,18 - 13,-5,NaN,0.0
9,0022300019,65,6,2,1,7:21 PM,7:05,Harris S.FOUL (P1.T2) (J.Goldenberg),None,None,None,None,Harris,1.0


In [ ]:
# Now create a column called 'player_subbed' which contains the player who was subbed out
# The format of the substitution is SUB: <last name> FOR <last name>
# An efficient way to do this is to only search the description where eventmsgtype is 8 (substitution)
# and then use the .str.split() method to get the player who was subbed out
substitution_rows = foul_plays[foul_plays['EVENTMSGTYPE'] == 8]
foul_plays['player_subbed'] = substitution_rows['HOMEDESCRIPTION'].str.extract(r'SUB: \w+ FOR (\w+)').fillna(
    substitution_rows['VISITORDESCRIPTION'].str.extract(r'SUB: \w+ FOR (\w+)')
)

foul_plays.head(15)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,fouler,foul_counts,player_subbed
0,0022300019,10,2,47,1,7:12 PM,11:16,MISS Embiid 6' Turnaround Jump Shot,None,None,None,None,NaN,0.0,NaN
1,0022300019,11,4,0,1,7:12 PM,11:16,None,None,Pacers Rebound,None,None,NaN,0.0,NaN
2,0022300019,12,6,3,1,7:12 PM,11:16,Embiid L.B.FOUL (P1.T1) (S.Wall),None,None,None,None,Embiid,1.0,NaN
3,0022300019,45,6,1,1,7:18 PM,8:19,None,None,Turner P.FOUL (P1.T1) (S.Wall),None,None,Turner,1.0,NaN
4,0022300019,51,6,2,1,7:19 PM,7:45,None,None,Turner S.FOUL (P2.T2) (J.Goldenberg),None,None,Turner,2.0,NaN
5,0022300019,53,3,11,1,7:20 PM,7:45,Embiid Free Throw 1 of 2 (1 PTS),None,None,14 - 9,-5,NaN,0.0,NaN
6,0022300019,54,8,0,1,7:20 PM,7:45,None,None,SUB: Smith FOR Turner,None,None,NaN,0.0,Turner
7,0022300019,56,3,12,1,7:20 PM,7:45,Embiid Free Throw 2 of 2 (2 PTS),None,None,14 - 10,-4,NaN,0.0,NaN
8,0022300019,63,1,98,1,7:21 PM,7:05,None,None,Smith 4' Cutting Layup Shot (2 PTS) (Haliburto...,18 - 13,-5,NaN,0.0,NaN
9,0022300019,65,6,2,1,7:21 PM,7:05,Harris S.FOUL (P1.T2) (J.Goldenberg),None,None,None,None,Harris,1.0,NaN


In [ ]:
# Now create a column called 'fouler_subbed' which is True if the player who committed the foul was subbed out
# Keep in mind when building this that the row values will not align since the foul will be assessed prior to the sub
# and the order is not predictable. Therefore, return True if at any point the player who committed the foul was subbed out
# as long as PERIOD and PCTIMESTRING are the same for both rows

# Reorganize a new df where the index is the time of the foul and the columns are the players who committed the foul/got subbed out
foul_plays_reorganized = foul_plays[['PERIOD', 'PCTIMESTRING', 'fouler', 'player_subbed']].set_index(['PERIOD', 'PCTIMESTRING'])
# Compress the PCTIMESTRING column to only contain one observation with everything being compressed into a list
foul_plays_reorganized = foul_plays_reorganized.groupby(['PERIOD', 'PCTIMESTRING']).agg(list)
# Now create a column called 'fouler_subbed' which is True if the player who committed the foul was subbed out
foul_plays_reorganized['fouler_subbed'] = foul_plays_reorganized.apply(
    lambda row: any([fouler in row['player_subbed'] for fouler in row['fouler']]), axis=1
)
# Now port the fouler_subbed column back to the original DataFrame, keeping in mind that the index is PERIOD and PCTIMESTRING
# so we need to reset the index and then merge the two DataFrames on PERIOD and PCTIMESTRING
foul_plays = foul_plays.reset_index().merge(foul_plays_reorganized[['fouler_subbed']], on=['PERIOD', 'PCTIMESTRING']).set_index('index')

foul_plays.head(10)


,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,fouler,foul_counts,player_subbed,fouler_subbed
index,,,,,,,,,,,,,,,,
0,0022300019,10,2,47,1,7:12 PM,11:16,MISS Embiid 6' Turnaround Jump Shot,None,None,None,None,NaN,0.0,NaN,False
1,0022300019,11,4,0,1,7:12 PM,11:16,None,None,Pacers Rebound,None,None,NaN,0.0,NaN,False
2,0022300019,12,6,3,1,7:12 PM,11:16,Embiid L.B.FOUL (P1.T1) (S.Wall),None,None,None,None,Embiid,1.0,NaN,False
3,0022300019,45,6,1,1,7:18 PM,8:19,None,None,Turner P.FOUL (P1.T1) (S.Wall),None,None,Turner,1.0,NaN,False
4,0022300019,51,6,2,1,7:19 PM,7:45,None,None,Turner S.FOUL (P2.T2) (J.Goldenberg),None,None,Turner,2.0,NaN,True
5,0022300019,53,3,11,1,7:20 PM,7:45,Embiid Free Throw 1 of 2 (1 PTS),None,None,14 - 9,-5,NaN,0.0,NaN,True
6,0022300019,54,8,0,1,7:20 PM,7:45,None,None,SUB: Smith FOR Turner,None,None,NaN,0.0,Turner,True
7,0022300019,56,3,12,1,7:20 PM,7:45,Embiid Free Throw 2 of 2 (2 PTS),None,None,14 - 10,-4,NaN,0.0,NaN,True
8,0022300019,63,1,98,1,7:21 PM,7:05,None,None,Smith 4' Cutting Layup Shot (2 PTS) (Haliburto...,18 - 13,-5,NaN,0.0,NaN,False


In [ ]:
# Now create a new column called time which is the time  in the game
# Period is 1-4, so multiply by 12 to get the number of minutes in the game
# Then add the number of minutes left in the period
# Then add the number of seconds left in the period
# Keep unit as minutes
foul_plays['time'] = (foul_plays['PERIOD'] - 1) * 12 + 12 - foul_plays['PCTIMESTRING'].str.split(':').apply(
    lambda row: float(row[0]) + float(row[1]) / 60
)

foul_plays.head(10)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,fouler,foul_counts,player_subbed,fouler_subbed,time
index,,,,,,,,,,,,,,,,,
0,0022300019,10,2,47,1,7:12 PM,11:16,MISS Embiid 6' Turnaround Jump Shot,None,None,None,None,NaN,0.0,NaN,False,0.733333
1,0022300019,11,4,0,1,7:12 PM,11:16,None,None,Pacers Rebound,None,None,NaN,0.0,NaN,False,0.733333
2,0022300019,12,6,3,1,7:12 PM,11:16,Embiid L.B.FOUL (P1.T1) (S.Wall),None,None,None,None,Embiid,1.0,NaN,False,0.733333
3,0022300019,45,6,1,1,7:18 PM,8:19,None,None,Turner P.FOUL (P1.T1) (S.Wall),None,None,Turner,1.0,NaN,False,3.683333
4,0022300019,51,6,2,1,7:19 PM,7:45,None,None,Turner S.FOUL (P2.T2) (J.Goldenberg),None,None,Turner,2.0,NaN,True,4.250000
5,0022300019,53,3,11,1,7:20 PM,7:45,Embiid Free Throw 1 of 2 (1 PTS),None,None,14 - 9,-5,NaN,0.0,NaN,True,4.250000
6,0022300019,54,8,0,1,7:20 PM,7:45,None,None,SUB: Smith FOR Turner,None,None,NaN,0.0,Turner,True,4.250000
7,0022300019,56,3,12,1,7:20 PM,7:45,Embiid Free Throw 2 of 2 (2 PTS),None,None,14 - 10,-4,NaN,0.0,NaN,True,4.250000
8,0022300019,63,1,98,1,7:21 PM,7:05,None,None,Smith 4' Cutting Layup Shot (2 PTS) (Haliburto...,18 - 13,-5,NaN,0.0,NaN,False,4.916667


In [ ]:
# Now get time since last foul
# Create a column called 'time_since_last_foul' which is the time since the last foul for the player who committed the foul
# Use the time column to get the time of the last foul for the player who committed the foul and subtract it from the current time
# Handle the case where there is no previous foul by returning the current time
foul_plays['time_since_last_foul'] = foul_plays.groupby('fouler')['time'].diff().fillna(foul_plays['time'])

foul_plays.head(10)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,fouler,foul_counts,player_subbed,fouler_subbed,time,time_since_last_foul
index,,,,,,,,,,,,,,,,,,
0,0022300019,10,2,47,1,7:12 PM,11:16,MISS Embiid 6' Turnaround Jump Shot,None,None,None,None,NaN,0.0,NaN,False,0.733333,0.733333
1,0022300019,11,4,0,1,7:12 PM,11:16,None,None,Pacers Rebound,None,None,NaN,0.0,NaN,False,0.733333,0.733333
2,0022300019,12,6,3,1,7:12 PM,11:16,Embiid L.B.FOUL (P1.T1) (S.Wall),None,None,None,None,Embiid,1.0,NaN,False,0.733333,0.733333
3,0022300019,45,6,1,1,7:18 PM,8:19,None,None,Turner P.FOUL (P1.T1) (S.Wall),None,None,Turner,1.0,NaN,False,3.683333,3.683333
4,0022300019,51,6,2,1,7:19 PM,7:45,None,None,Turner S.FOUL (P2.T2) (J.Goldenberg),None,None,Turner,2.0,NaN,True,4.250000,0.566667
5,0022300019,53,3,11,1,7:20 PM,7:45,Embiid Free Throw 1 of 2 (1 PTS),None,None,14 - 9,-5,NaN,0.0,NaN,True,4.250000,4.250000
6,0022300019,54,8,0,1,7:20 PM,7:45,None,None,SUB: Smith FOR Turner,None,None,NaN,0.0,Turner,True,4.250000,4.250000
7,0022300019,56,3,12,1,7:20 PM,7:45,Embiid Free Throw 2 of 2 (2 PTS),None,None,14 - 10,-4,NaN,0.0,NaN,True,4.250000,4.250000
8,0022300019,63,1,98,1,7:21 PM,7:05,None,None,Smith 4' Cutting Layup Shot (2 PTS) (Haliburto...,18 - 13,-5,NaN,0.0,NaN,False,4.916667,4.916667


In [ ]:
# Now clean the data to match structure for easy analysis
# Index the DataFrame by the time of the foul (PERIOD and PCTIMESTRING)
# Keep fouler and fouler_subbed columns, foul_counts, SCOREMARGIN, time, time_since_last_foul (for now)
keep = ['PERIOD', 'PCTIMESTRING', 'fouler', 'fouler_subbed', 'foul_counts', 'SCOREMARGIN', 'time', 'time_since_last_foul']
structured_foul_plays = foul_plays[keep].set_index(['PERIOD', 'PCTIMESTRING'])
# Drop rows where fouler is NaN
structured_foul_plays = structured_foul_plays.dropna(subset=['fouler'])
# Rework the index where there is a new index called time represnted as Period:PCTIMESTRING
structured_foul_plays.index = structured_foul_plays.index.map(lambda x: str(x[0]) + ':' + x[1])

structured_foul_plays.head(25)

,fouler,fouler_subbed,foul_counts,SCOREMARGIN,time,time_since_last_foul
1:11:16,Embiid,False,1.0,None,0.733333,0.733333
1:8:19,Turner,False,1.0,None,3.683333,3.683333
1:7:45,Turner,True,2.0,None,4.250000,0.566667
1:7:05,Harris,False,1.0,None,4.916667,4.916667
1:6:30,Smith,False,1.0,None,5.500000,5.500000
1:5:42,Smith,True,2.0,None,6.300000,0.800000
1:4:44,Morris,False,1.0,None,7.266667,7.266667
1:2:37,McConnell,False,1.0,None,9.383333,9.383333
1:1:26,Jackson,False,1.0,None,10.566667,10.566667
1:0:12,Jackson,False,2.0,None,11.800000,1.233333


Super Basic Analysis

In [ ]:
# Print the probability of a player being subbed out after committing a foul
print(f'Probability of a player being subbed out after committing a foul: {structured_foul_plays["fouler_subbed"].mean()}')

# Print the probability of a player being subbed out after committing n fouls
for i in range(1, 6):
    print(f'Probability of a player being subbed out after committing {i} fouls: {structured_foul_plays[structured_foul_plays["foul_counts"] == i]["fouler_subbed"].mean()}')

Probability of a player being subbed out after committing a foul: 0.1875
Probability of a player being subbed out after committing 1 fouls: 0.058823529411764705
Probability of a player being subbed out after committing 2 fouls: 0.3076923076923077
Probability of a player being subbed out after committing 3 fouls: 0.125
Probability of a player being subbed out after committing 4 fouls: 0.3333333333333333
Probability of a player being subbed out after committing 5 fouls: 0.0


# Time To Scale the Procedure To Record Data For Every Single Game

In [ ]:
# First get every game ID from the season
games = leaguegamefinder.LeagueGameFinder(season_nullable=Season.default,
                                          season_type_nullable=SeasonType.regular).get_normalized_dict()['LeagueGameFinderResults']
game_ids = set([game['GAME_ID'] for game in games])
print(f'Number of games: {len(game_ids)}')

Number of games: 179


Now I Need To Make Some Sort Of Object To Format the Data Following the Early Single Game Procedure

In [ ]:
class FormatData:
    def __init__(self, df):
        self.df = df
        self.output = None
        

    def get_foul_data(self):
        # Create a DataFrame of what happens at the time of each foul, so when a foul occurs, 
        # see what else happens at the same PERIOD and PCTIMESTRING and store it in a DataFrame
        # Extract rows where EVENTMSGTYPE is 6 (foul events)
        foul_times = self.df[self.df['EVENTMSGTYPE'] == 6][['PERIOD', 'PCTIMESTRING']]

        # Merge the original DataFrame with the foul_times DataFrame on 'PERIOD' and 'PCTIMESTRING'
        foul_plays = pd.merge(self.df, foul_times, on=['PERIOD', 'PCTIMESTRING'], how='inner')

        self.df = foul_plays
        pass

    def add_fouler(self):
        # Create a column called fouler which contains the player who committed the foul
        # Fill that by seeing the text that comes before the word '<char>.FOUL' in the 'HOMEDESCRIPTION' 
        # and 'VISITORDESCRIPTION' columns
        foul_plays = self.df
        foul_plays['fouler'] = np.where(foul_plays['HOMEDESCRIPTION'].str.contains('.FOUL'), 
                                        foul_plays['HOMEDESCRIPTION'].str.split(' ').str[0], 
                                        np.where(foul_plays['VISITORDESCRIPTION'].str.contains('.FOUL'), 
                                                foul_plays['VISITORDESCRIPTION'].str.split(' ').str[0], 
                                                np.nan))
        self.df = foul_plays
        pass

    def add_foul_counts(self):
        # Create a foul_counts DataFrame which contains the number of fouls each player committed
        # Example: Turner S.FOUL (P2.T2) (J.Goldenberg) here Turner committed a foul
        # he has 2 fouls (because P2) and the team has 2 fouls (because T2)
        foul_plays = self.df
        foul_plays['foul_counts'] = foul_plays[['HOMEDESCRIPTION', 'VISITORDESCRIPTION']].apply(
            lambda row: row.str.extract(r'\(P(\d+)\.').astype(float).sum(), axis=1
        )
        self.df = foul_plays
        pass

    def add_player_subbed(self):
        # Now create a column called 'player_subbed' which contains the player who was subbed out
        # The format of the substitution is SUB: <last name> FOR <last name>
        # An efficient way to do this is to only search the description where eventmsgtype is 8 (substitution)
        # and then use the .str.split() method to get the player who was subbed out
        foul_plays = self.df
        substitution_rows = foul_plays[foul_plays['EVENTMSGTYPE'] == 8]
        foul_plays['player_subbed'] = substitution_rows['HOMEDESCRIPTION'].str.extract(r'SUB: \w+ FOR (\w+)').fillna(
            substitution_rows['VISITORDESCRIPTION'].str.extract(r'SUB: \w+ FOR (\w+)')
        )
        self.df = foul_plays
        pass

    def add_fouler_subbed(self):
        # Now create a column called 'fouler_subbed' which is True if the player who committed the foul was subbed out
        # Keep in mind when building this that the row values will not align since the foul will be assessed prior to the sub
        # and the order is not predictable. Therefore, return True if at any point the player who committed the foul was subbed out
        # as long as PERIOD and PCTIMESTRING are the same for both rows
        foul_plays = self.df
        # Reorganize a new df where the index is the time of the foul and the columns are the players who committed the foul/got subbed out
        foul_plays_reorganized = foul_plays[['PERIOD', 'PCTIMESTRING', 'fouler', 'player_subbed']].set_index(['PERIOD', 'PCTIMESTRING'])
        # Compress the PCTIMESTRING column to only contain one observation with everything being compressed into a list
        foul_plays_reorganized = foul_plays_reorganized.groupby(['PERIOD', 'PCTIMESTRING']).agg(list)
        # Now create a column called 'fouler_subbed' which is True if the player who committed the foul was subbed out
        foul_plays_reorganized['fouler_subbed'] = foul_plays_reorganized.apply(
            lambda row: any([fouler in row['player_subbed'] for fouler in row['fouler']]), axis=1
        )
        # Now port the fouler_subbed column back to the original DataFrame, keeping in mind that the index is PERIOD and PCTIMESTRING
        # so we need to reset the index and then merge the two DataFrames on PERIOD and PCTIMESTRING
        foul_plays = foul_plays.reset_index().merge(foul_plays_reorganized[['fouler_subbed']], on=['PERIOD', 'PCTIMESTRING']).set_index('index')
        self.df = foul_plays
        pass

    def add_time(self):
        # Now create a new column called time which is the time  in the game
        # Period is 1-4, so multiply by 12 to get the number of minutes in the game
        # Then add the number of minutes left in the period
        # Then add the number of seconds left in the period
        # Keep unit as minutes
        foul_plays = self.df
        foul_plays['time'] = (foul_plays['PERIOD'] - 1) * 12 + 12 - foul_plays['PCTIMESTRING'].str.split(':').apply(
            lambda row: float(row[0]) + float(row[1]) / 60
        )
        self.df = foul_plays
        pass

    def add_time_since_last_foul(self):
        # Now get time since last foul
        # Create a column called 'time_since_last_foul' which is the time since the last foul for the player who committed the foul
        # Use the time column to get the time of the last foul for the player who committed the foul and subtract it from the current time
        # Handle the case where there is no previous foul by returning the current time
        foul_plays = self.df
        foul_plays['time_since_last_foul'] = foul_plays.groupby('fouler')['time'].diff().fillna(foul_plays['time'])
        self.df = foul_plays
        pass

    def clean_data(self):
        # Now clean the data to match structure for easy analysis
        # Index the DataFrame by the time of the foul (PERIOD and PCTIMESTRING)
        # Keep fouler and fouler_subbed columns, foul_counts, SCOREMARGIN, time, time_since_last_foul (for now)
        foul_plays = self.df
        keep = ['PERIOD', 'PCTIMESTRING', 'fouler', 'fouler_subbed', 'foul_counts', 'SCOREMARGIN', 'time', 'time_since_last_foul']
        structured_foul_plays = foul_plays[keep].set_index(['PERIOD', 'PCTIMESTRING'])
        # Drop rows where fouler is NaN
        structured_foul_plays = structured_foul_plays.dropna(subset=['fouler'])
        # Rework the index where there is a new index called time represnted as Period:PCTIMESTRING
        structured_foul_plays.index = structured_foul_plays.index.map(lambda x: str(x[0]) + ':' + x[1])
        self.df = structured_foul_plays
        pass

    def apply_pipeline(self):
        self.get_foul_data()
        self.add_fouler()
        self.add_foul_counts()
        self.add_player_subbed()
        self.add_fouler_subbed()
        self.add_time()
        self.add_time_since_last_foul()
        self.clean_data()
        self.output = self.df

In [ ]:
# Test the class
df = playbyplay.PlayByPlay(game_ids[-2]).get_data_frames()[0]
test = FormatData(df)
test.apply_pipeline()
test.output.head(10)

,fouler,fouler_subbed,foul_counts,SCOREMARGIN,time,time_since_last_foul
1:9:28,Ayton,False,1.0,None,2.533333,2.533333
1:9:10,Thybulle,False,1.0,None,2.833333,2.833333
1:7:19,Henderson,False,1.0,None,4.683333,4.683333
1:4:52,Henderson,False,2.0,None,7.133333,2.450000
1:3:40,Beverley,False,1.0,None,8.333333,8.333333
1:1:17,Beverley,False,2.0,None,10.716667,2.383333
1:0:12,Sharpe,False,1.0,None,11.800000,11.800000
1:0:02,Embiid,False,1.0,None,11.966667,11.966667
2:11:19,Reed,False,1.0,None,12.683333,12.683333
2:10:41,Camara,False,1.0,None,13.316667,13.316667


In [ ]:
# Continue test by adding another game to it
df = playbyplay.PlayByPlay(game_ids[-3]).get_data_frames()[0]
test2 = FormatData(df)
test2.apply_pipeline()
test2.output.head(10)

,fouler,fouler_subbed,foul_counts,SCOREMARGIN,time,time_since_last_foul
1:10:31,Collins,False,1.0,None,1.483333,1.483333
1:9:50,Sochan,False,1.0,None,2.166667,2.166667
1:9:42,Toppin,False,1.0,None,2.300000,2.300000
1:9:22,Sochan,True,2.0,None,2.633333,0.466667
1:8:45,Mathurin,False,1.0,None,3.250000,3.250000
1:7:07,Wembanyama,False,1.0,None,4.883333,4.883333
1:6:00,Turner,False,1.0,None,6.000000,6.000000
1:5:27,Johnson,True,1.0,None,6.550000,6.550000
1:1:49,Jones,False,1.0,None,10.183333,10.183333
1:1:38,Smith,False,1.0,None,10.366667,10.366667


In [ ]:
# Now combine test and test2
# Create a new DataFrame called combined which is the concatenation of test and test2
# The index is game_id:<current index>
# The current index is the index of the DataFrame
# The game_id is the game_id of the DataFrame
combined = pd.concat([test.output, test2.output], keys=[game_ids[-2], game_ids[-3]], names=['game_id', 'index'])
combined.head(-10)

fouler  fouler_subbed  foul_counts SCOREMARGIN  \
game_id    index                                                       
0022300098 1:9:28      Ayton          False          1.0        None   
           1:9:10   Thybulle          False          1.0        None   
           1:7:19  Henderson          False          1.0        None   
           1:4:52  Henderson          False          2.0        None   
           1:3:40   Beverley          False          1.0        None   
...                      ...            ...          ...         ...   
0022300146 2:3:50      Smith           True          3.0        None   
           2:3:27    Collins           True          2.0        None   
           2:1:51      Brown          False          1.0        None   
           2:1:12    Jackson          False          3.0        None   
           3:7:23     Toppin          False          3.0        None   

                        time  time_since_last_foul  
game_id    index                                    
0022300098 1:9:28   2.533333              2.533333  
           1:9:10   2.833333              2.833333  
           1:7:19   4.683333              4.683333  
           1:4:52   7.133333              2.450000  
           1:3:40   8.333333              8.333333  
...                      ...                   ...  
0022300146 2:3:50  20.166667              8.466667  
           2:3:27  20.550000             19.066667  
           2:1:51  22.150000             22.150000  
           2:1:12  22.800000              8.883333  
           3:7:23  28.616667             26.316667  

[71 rows x 6 columns]

Now We Know that the DFs Format, Put It All Together

In [ ]:
# First Initalize the dataframe
all_data = pd.DataFrame()

In [ ]:
# Initialize an empty list to store all the dataframes
dataframes = []

# Loop through all the game_ids
for game_id in game_ids:
    # Get the data
    df = playbyplay.PlayByPlay(game_id).get_data_frames()[0]
    
    # Format the data
    formatted = FormatData(df)
    
    # Apply the pipeline
    formatted.apply_pipeline()
    
    # Append the output dataframe to the list
    dataframes.append(formatted.output)

    # Sleep for a random amount of time between 1 and 3 seconds to avoid too many requests
    time.sleep(np.random.uniform(5, 15))

# Concatenate all the dataframes in the list into a single dataframe
all_data = pd.concat(dataframes, keys=game_ids, names=['game_id', 'index'])

all_data.head(10)

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [6]:
# See how many games have been pulled
print(f'Number of games pulled: {len(all_data.index.get_level_values(0).unique())}')

NameError: name 'all_data' is not defined

In [ ]:
# Print the probability of a player being subbed out after committing a foul
print(f'Probability of a player being subbed out after committing a foul: {all_data["fouler_subbed"].mean()}')

# Print the probability of a player being subbed out after committing n fouls
for i in range(1, 6):
    print(f'Probability of a player being subbed out after committing {i} fouls: {all_data[all_data["foul_counts"] == i]["fouler_subbed"].mean()}')

Probability of a player being subbed out after committing a foul: 0.11475409836065574
Probability of a player being subbed out after committing 1 fouls: 0.07936507936507936
Probability of a player being subbed out after committing 2 fouls: 0.08823529411764706
Probability of a player being subbed out after committing 3 fouls: 0.23529411764705882
Probability of a player being subbed out after committing 4 fouls: 0.2857142857142857
Probability of a player being subbed out after committing 5 fouls: 0.0
